# 🔄 Normalizing Flow Model in PyTorch

This notebook provides a simple implementation of a normalizing flow model.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader
import matplotlib.pyplot as plt

In [ ]:
from sklearn.datasets import make_moons

# Generate two moons data
X, y = make_moons(n_samples=10000, noise=0.05)
X = torch.tensor(X, dtype=torch.float32)

# Visualize (optional)
plt.scatter(X[:, 0], X[:, 1], s=5, alpha=0.6)
plt.title("Two Moons Dataset")
plt.show()

# Create PyTorch dataset and dataloader
dataset = TensorDataset(X)
dataloader = DataLoader(dataset, batch_size=256, shuffle=True)

## 🧱 Define the Affine Coupling Layer

In [ ]:
class AffineCoupling(nn.Module):
    def __init__(self, dim, hidden_dim=128):
        super().__init__()
        self.scale_net = nn.Sequential(
            nn.Linear(dim // 2, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, dim // 2),
            nn.Tanh()
        )
        self.shift_net = nn.Sequential(
            nn.Linear(dim // 2, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, dim // 2)
        )

    def forward(self, x):
        x1, x2 = x.chunk(2, dim=1)
        s = self.scale_net(x1)
        t = self.shift_net(x1)
        y2 = x2 * torch.exp(s) + t
        y = torch.cat([x1, y2], dim=1)
        log_det = s.sum(dim=1)
        return y, log_det

    def inverse(self, y):
        y1, y2 = y.chunk(2, dim=1)
        s = self.scale_net(y1)
        t = self.shift_net(y1)
        x2 = (y2 - t) * torch.exp(-s)
        x = torch.cat([y1, x2], dim=1)
        log_det = -s.sum(dim=1)
        return x, log_det

## 🔁 Build the Flow Model

In [ ]:
class NormalizingFlowModel(nn.Module):
    def __init__(self, dim, n_flows):
        super().__init__()
        self.dim = dim
        self.base_dist = torch.distributions.MultivariateNormal(
            torch.zeros(dim), torch.eye(dim)
        )
        self.flows = nn.ModuleList([AffineCoupling(dim) for _ in range(n_flows)])

    def forward(self, x):
        log_det_total = torch.zeros(x.size(0))
        for flow in self.flows:
            x, log_det = flow(x)
            log_det_total += log_det
        log_prob = self.base_dist.log_prob(x) + log_det_total
        return log_prob

    def sample(self, n_samples):
        z = self.base_dist.sample((n_samples,))
        for flow in reversed(self.flows):
            z, _ = flow.inverse(z)
        return z

## 🔍 Example: Sampling and Density Evaluation

In [ ]:
dim = 4
n_flows = 4
flow_model = NormalizingFlowModel(dim, n_flows)

# Evaluate log-likelihood
x = torch.randn(10, dim)
log_probs = flow_model(x)
print("Log-likelihoods:", log_probs)

# Sample from model
samples = flow_model.sample(500).detach().numpy()

# Visualize first two dimensions
plt.figure(figsize=(6, 5))
plt.scatter(samples[:, 0], samples[:, 1], alpha=0.6)
plt.title("Samples from Flow Model (dim 0 vs 1)")
plt.xlabel("x0")
plt.ylabel("x1")
plt.grid(True)
plt.show()